In [6]:
import pandas as pd
df = pd.read_csv('./201707-citibike-tripdata.csv')
df

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
0,364,2017-07-01 00:00:00,2017-07-01 00:06:05,539,Metropolitan Ave & Bedford Ave,40.715348,-73.960241,3107,Bedford Ave & Nassau Ave,40.723117,-73.952123,14744,Subscriber,1986.0,1
1,2142,2017-07-01 00:00:03,2017-07-01 00:35:46,293,Lafayette St & E 8 St,40.730207,-73.991026,3425,2 Ave & E 104 St,40.789210,-73.943708,19587,Subscriber,1981.0,1
2,328,2017-07-01 00:00:08,2017-07-01 00:05:37,3242,Schermerhorn St & Court St,40.691029,-73.991834,3397,Court St & Nelson St,40.676395,-73.998699,27937,Subscriber,1984.0,2
3,2530,2017-07-01 00:00:11,2017-07-01 00:42:22,2002,Wythe Ave & Metropolitan Ave,40.716887,-73.963198,398,Atlantic Ave & Furman St,40.691652,-73.999979,26066,Subscriber,1985.0,1
4,2534,2017-07-01 00:00:15,2017-07-01 00:42:29,2002,Wythe Ave & Metropolitan Ave,40.716887,-73.963198,398,Atlantic Ave & Furman St,40.691652,-73.999979,29408,Subscriber,1982.0,2
5,545,2017-07-01 00:00:18,2017-07-01 00:09:24,361,Allen St & Hester St,40.716059,-73.991908,502,Henry St & Grand St,40.714215,-73.981346,21203,Subscriber,1954.0,2
6,1496,2017-07-01 00:00:18,2017-07-01 00:25:15,3233,E 48 St & 5 Ave,40.757246,-73.978059,546,E 30 St & Park Ave S,40.744449,-73.983035,15933,Customer,1971.0,1
7,1495,2017-07-01 00:00:19,2017-07-01 00:25:15,3233,E 48 St & 5 Ave,40.757246,-73.978059,546,E 30 St & Park Ave S,40.744449,-73.983035,18515,Customer,1964.0,2
8,593,2017-07-01 00:00:27,2017-07-01 00:10:20,3314,W 95 St & Broadway,40.793770,-73.971888,3363,E 102 St & Park Ave,40.790483,-73.950331,29135,Subscriber,1982.0,1
9,1950,2017-07-01 00:00:30,2017-07-01 00:33:00,3085,Roebling St & N 4 St,40.714690,-73.957390,468,Broadway & W 56 St,40.765265,-73.981923,17365,Subscriber,1983.0,1


In [44]:
def miss():
    col = pd.isnull(df['birth year']).tolist()
    assert len(col) == 1735599
    cnt = 0
    for i, item in enumerate(col):
        if item == True:
            cnt += 1
    print float(cnt) / float(len(col))

In [51]:
df['birth year'] = df['birth year'].fillna(df['birth year'].mean())
df['birth year'].mean()

1979.5916073159776

In [45]:
miss()

0.0


In [15]:
col = df['tripduration'].tolist()
col.sort()
print len(col)
col

1735599


[61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,
 61,


In [50]:
col = df['start station id'].tolist() + df['end station id'].tolist()
col = set(col)
len(col)

634

In [82]:
start_station = df[['start station id', 'start station name', 'start station latitude', 'start station longitude']]
start_station.columns = ['station id', 'station name', 'station latitude', 'station logitude']
end_station = df[['end station id', 'end station name', 'end station latitude', 'end station longitude']]
end_station.columns = ['station id', 'station name', 'station latitude', 'station logitude']
station = start_station.append(end_station, ignore_index = True)
station

,station id,station name,station latitude,station logitude
0,539,Metropolitan Ave & Bedford Ave,40.715348,-73.960241
1,293,Lafayette St & E 8 St,40.730207,-73.991026
2,3242,Schermerhorn St & Court St,40.691029,-73.991834
3,2002,Wythe Ave & Metropolitan Ave,40.716887,-73.963198
4,2002,Wythe Ave & Metropolitan Ave,40.716887,-73.963198
5,361,Allen St & Hester St,40.716059,-73.991908
6,3233,E 48 St & 5 Ave,40.757246,-73.978059
7,3233,E 48 St & 5 Ave,40.757246,-73.978059
8,3314,W 95 St & Broadway,40.793770,-73.971888
9,3085,Roebling St & N 4 St,40.714690,-73.957390


In [87]:
station = station.drop_duplicates()
station = station.reset_index(drop = True)
station_num = len(station)
station

,station id,station name,station latitude,station logitude
0,539,Metropolitan Ave & Bedford Ave,40.715348,-73.960241
1,293,Lafayette St & E 8 St,40.730207,-73.991026
2,3242,Schermerhorn St & Court St,40.691029,-73.991834
3,2002,Wythe Ave & Metropolitan Ave,40.716887,-73.963198
4,361,Allen St & Hester St,40.716059,-73.991908
5,3233,E 48 St & 5 Ave,40.757246,-73.978059
6,3314,W 95 St & Broadway,40.793770,-73.971888
7,3085,Roebling St & N 4 St,40.714690,-73.957390
8,382,University Pl & E 14 St,40.734927,-73.992005
9,305,E 58 St & 3 Ave,40.760958,-73.967245


In [98]:
from geopy.distance import great_circle
dist_sum = 0
for i in xrange(station_num):
    for j in xrange(station_num):
        if i != j:
            a = (station.iloc[i, 2], station.iloc[i, 3])
            b = (station.iloc[j, 2], station.iloc[j, 3])
            dist_sum += great_circle(a, b).meters
print dist_sum / float(station_num * (station_num - 1) / 2)

10877.7798769
